In [1]:
import pandas as pd
import os
import nltk
import numpy as np
import pandas as pd
from collections import Counter
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import import_ipynb
from config import Config
from utils import pickle_dump

importing Jupyter notebook from config.ipynb
importing Jupyter notebook from utils.ipynb


In [2]:
df = pd.read_csv('input_data/input.csv')
train = df.sample(frac=0.7)
test = df.loc[~df.index.isin(train.index)]

In [3]:
train.drop(train.columns[0],axis=1,inplace=True)
test.drop(test.columns[0],axis=1,inplace=True)
train.to_csv('data/train.csv',index=False)
test.to_csv('data/test.csv',index=False)
test

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,sentiment,question,aspect,from,to
0,1,whether government is aware that there is spur...,last,88,92
5,1,the details of central universities which have...,universities,23,35
8,1,the total number of central universities which...,universities,28,40
16,1,whether government is aware that most of the p...,drinking,54,62
21,1,whether it is a fact that the method adopted b...,banks,48,53
23,1,whether any proposal of merger of anyonya bank...,bank,42,46
24,1,whether it is a fact that physiotherapists hav...,nursing,101,108
26,1,the details of the commission structure of int...,last,170,174
27,2,the details of commitment charges paid to diff...,last,173,177
29,2,whether it is a fact that the state government...,credit,130,136


In [4]:
def load_glove(filename):
    word_vec = {}
    emd_dim = -1
    with open(filename, 'r', encoding = "utf-8") as f:
        for line in f:
            line = line.strip().split()
            word = line[0]
            word_vector = np.array([float(v) for v in line[1:]])
            word_vec[word] = word_vector
            if emd_dim == -1:
                emd_dim = len(word_vector)

    assert (len(vw) == emd_dim for vw in word_vec.values())

    return word_vec, emd_dim

In [5]:
def flat_list(l):
    result = list()
    for item in l:
        if isinstance(item, (list, tuple)):
            result.extend(item)
        else:
            result.append(item)
    return result

In [6]:
def build_vocabulary(corpus, start_id=1):
    corpus = flat_list(corpus)
    return dict((word, idx) for idx, word in enumerate(set(corpus), start=start_id))

In [7]:
def build_embedding(corpus, vocab, embedding_dim=300):
    model = Word2Vec(corpus, size=embedding_dim, min_count=1, window=5, sg=1, iter=10)
    weights = model.wv.syn0
    d = dict([(k, v.index) for k, v in model.wv.vocab.items()])
    emb = np.zeros(shape=(len(vocab) + 2, embedding_dim), dtype='float32')
    count = 0
    for w, i in vocab.items():
        if w not in d:
            count += 1
            emb[i, :] = np.random.uniform(-0.1, 0.1, embedding_dim)
        else:
            emb[i, :] = weights[d[w], :]
    print('embedding out of vocabulary：', count)
    return emb

In [8]:
def build_glove_embedding(vocab, word_vectors, embed_dim):
    emb_matrix = np.zeros(shape=(len(vocab) + 2, embed_dim), dtype='float32')

    count = 0
    for word, i in vocab.items():
        if word not in word_vectors:
            count += 1
            emb_matrix[i, :] = np.random.uniform(-0.1, 0.1, embed_dim)
        else:
            emb_matrix[i, :] = word_vectors[word]
    print('glove embedding out of vocabulary：', count)
    return emb_matrix

In [9]:
def build_asp_embedding(aspect_vocab, split_func, word_vocab, word_embed):
    asp_emb = np.random.uniform(-0.1, 0.1, [len(aspect_vocab.keys()), word_embed.shape[1]])
    count = 0
    for aspect, aspect_id in aspect_vocab.items():
        word_ids = [word_vocab.get(word, 0) for word in split_func(aspect)]
        if any(word_ids):
            avg_vector = np.mean(word_embed[word_ids], axis=0)
            asp_emb[aspect_id] = avg_vector
        else:
            count += 1
    print('aspect embedding out of vocabulary:', count)
    return asp_emb

In [10]:
def build_asp_text_embedding(aspect_text_vocab, word_vocab, word_embed):
    asp_text_embed = np.zeros(shape=(len(aspect_text_vocab) + 2, word_embed.shape[1]), dtype='float32')
    count = 0
    for aspect, aspect_id in aspect_text_vocab.items():
        if aspect in word_vocab:
            asp_text_embed[aspect_id] = word_embed[word_vocab[aspect]]
        else:
            count += 1
            asp_text_embed[aspect_id] = np.random.uniform(-0.1, 0.1, word_embed.shape[1])
    print('aspect text embedding out of vocabulary:', count)
    return asp_text_embed

In [11]:
def analyze_len_distribution(train_input, test_input):
    text_len = list()
    text_len.extend([len(l) for l in train_input])
    text_len.extend([len(l) for l in test_input])
    max_len = np.max(text_len)
    min_len = np.min(text_len)
    avg_len = np.average(text_len)
    median_len = np.median(text_len)
    print('max len:', max_len, 'min_len', min_len, 'avg len', avg_len, 'median len', median_len)
    for i in range(int(median_len), int(max_len), 5):
        less = list(filter(lambda x: x <= i, text_len))
        ratio = len(less) / len(text_len)
        print(i, ratio)
        if ratio >= 0.99:
            break

In [12]:
def analyze_class_distribution(labels):
    for cls, count in Counter(labels).most_common():
        print(cls, count, count / len(labels))

In [13]:
def get_loc_info(l, start, end):
    pos_info = []
    offset_info =[]
    for i in range(len(l)):
        if i < start:
            pos_info.append(1 - abs(i - start) / len(l))
            offset_info.append(i - start)
        elif start <= i < end:
            pos_info.append(1.)
            offset_info.append(0.)
        else:
            pos_info.append(1 - abs(i - end + 1) / len(l))
            offset_info.append(i - end +1)
    return pos_info, offset_info

In [14]:
def split_and_get_loc_info(data, word_vocab, char_vocab, word_cut_func):
    word_input_l, word_input_r, word_input_r_with_pad, word_pos_input, word_offset_input = [], [], [], [], []
    char_input_l, char_input_r, char_input_r_with_pad, char_pos_input, char_offset_input = [], [], [], [], []
    word_mask, char_mask = [], []
    for idx, row in data.iterrows():
        text, word_list, char_list, aspect = row['question'], row['word_list'], row['char_list'], row['aspect']
        start, end = row['from'], row['to']

        char_input_l.append(list(map(lambda x: char_vocab.get(x, len(char_vocab)+1), char_list[:end])))
        char_input_r.append(list(map(lambda x: char_vocab.get(x, len(char_vocab)+1), char_list[start:])))
        char_input_r_with_pad.append([char_vocab.get(char_list[i], len(char_vocab)+1) if i >= start else 0
                                      for i in range(len(char_list))])  # replace left sequence with 0
        _char_mask = [1] * len(char_list)
        _char_mask[start:end] = [0.5] * (end-start)     # 1 for context, 0.5 for aspect
        char_mask.append(_char_mask)
        _pos_input, _offset_input = get_loc_info(char_list, start, end)
        char_pos_input.append(_pos_input)
        char_offset_input.append(_offset_input)

        word_list_l = word_cut_func(text[:start])
        word_list_r = word_cut_func(text[end:])
        start = len(word_list_l)
        end = len(word_list) - len(word_list_r)
        if word_list[start:end] != word_cut_func(aspect):
            if word_list[start-1:end] == word_cut_func(aspect):
                start -= 1
            elif word_list[start:end+1] == word_cut_func(aspect):
                end += 1
                print("Found -----------------------------------")
            else:
                print(('Can not find aspect `{}` in `{}`, word list : `{}`'.format(aspect, text, word_list)))
                #raise Exception('Can not find aspect `{}` in `{}`, word list : `{}`'.format(aspect, text, word_list))
        word_input_l.append(list(map(lambda x: word_vocab.get(x, len(word_vocab)+1), word_list[:end])))
        word_input_r.append(list(map(lambda x: word_vocab.get(x, len(word_vocab)+1), word_list[start:])))
        word_input_r_with_pad.append([word_vocab.get(word_list[i], len(word_vocab) + 1) if i >= start else 0
                                      for i in range(len(word_list))])     
        _word_mask = [1] * len(word_list)
        _word_mask[start:end] = [0.5] * (end - start)  
        word_mask.append(_word_mask)
        _pos_input, _offset_input = get_loc_info(word_list, start, end)
        word_pos_input.append(_pos_input)
        word_offset_input.append(_offset_input)
    return (word_input_l, word_input_r, word_input_r_with_pad, word_mask, word_pos_input, word_offset_input,
            char_input_l, char_input_r, char_input_r_with_pad, char_mask, char_pos_input, char_offset_input)

In [15]:
def pre_process(folder, word_cut_func, is_en):
    print('preprocessing: ', folder)
    train_data = pd.read_csv(os.path.join(folder, 'train.csv'), header=0, index_col=None)
    train_data['word_list'] = train_data['question'].apply(word_cut_func)
    train_data['char_list'] = train_data['question'].apply(lambda x: list(x))
    train_data['aspect_word_list'] = train_data['aspect'].apply(word_cut_func)
    train_data['aspect_char_list'] = train_data['aspect'].apply(lambda x: list(x))

    test_data = pd.read_csv(os.path.join(folder, 'test.csv'), header=0, index_col=None)
    test_data['word_list'] = test_data['question'].apply(word_cut_func)
    test_data['char_list'] = test_data['question'].apply(lambda x: list(x))
    test_data['aspect_word_list'] = test_data['aspect'].apply(word_cut_func)
    test_data['aspect_char_list'] = test_data['aspect'].apply(lambda x: list(x))

    print('size of training set:', len(train_data))
    print('size of test set:', len(test_data))

    word_corpus = np.concatenate((train_data['word_list'].values, 
                                  test_data['word_list'].values)).tolist()
    char_corpus = np.concatenate((train_data['char_list'].values,
                                  test_data['char_list'].values)).tolist()
    aspect_corpus = np.concatenate((train_data['aspect'].values, 
                                    test_data['aspect'].values)).tolist()
    aspect_text_word_corpus = np.concatenate((train_data['aspect_word_list'].values,
                                              test_data['aspect_word_list'].values)).tolist()
    aspect_text_char_corpus = np.concatenate((train_data['aspect_char_list'].values,
                                              test_data['aspect_char_list'].values)).tolist()

    # build vocabulary
    print('building vocabulary...')
    word_vocab = build_vocabulary(word_corpus, start_id=1)
    char_vocab = build_vocabulary(char_corpus, start_id=1)
    aspect_vocab = build_vocabulary(aspect_corpus, start_id=0)
    aspect_text_word_vocab = build_vocabulary(aspect_text_word_corpus, start_id=1)
    aspect_text_char_vocab = build_vocabulary(aspect_text_char_corpus, start_id=1)
    pickle_dump(word_vocab, os.path.join(folder, 'word_vocab.pkl'))
    pickle_dump(char_vocab, os.path.join(folder, 'char_vocab.pkl'))
    pickle_dump(aspect_vocab, os.path.join(folder, 'aspect_vocab.pkl'))
    pickle_dump(aspect_text_word_vocab, os.path.join(folder, 'aspect_text_word_vocab.pkl'))
    pickle_dump(aspect_text_char_vocab, os.path.join(folder, 'aspect_text_char_vocab.pkl'))
    print('finished building vocabulary!')
    print('len of word vocabulary:', len(word_vocab))
    print('sample of word vocabulary:', list(word_vocab.items())[:10])
    print('len of char vocabulary:', len(char_vocab))
    print('sample of char vocabulary:', list(char_vocab.items())[:10])
    print('len of aspect vocabulary:', len(aspect_vocab))
    print('sample of aspect vocabulary:', list(aspect_vocab.items())[:10])
    print('len of aspect text word vocabulary:', len(aspect_text_word_vocab))
    print('sample of aspect text word vocabulary:', list(aspect_text_word_vocab.items())[:10])
    print('len of aspect text char vocabulary:', len(aspect_text_char_vocab))
    print('sample of aspect text char vocabulary:', list(aspect_text_char_vocab.items())[:10])

    # prepare embedding
    print('preparing embedding...')
    word_w2v = build_embedding(word_corpus, word_vocab, config.word_embed_dim)
    aspect_word_w2v = build_asp_embedding(aspect_vocab, word_cut_func, word_vocab, word_w2v)
    aspect_text_word_w2v = build_asp_text_embedding(aspect_text_word_vocab, word_vocab, word_w2v)
    char_w2v = build_embedding(char_corpus, char_vocab, config.word_embed_dim)
    aspect_char_w2v = build_asp_embedding(aspect_vocab, lambda x: list(x), char_vocab, char_w2v)
    aspect_text_char_w2v = build_asp_text_embedding(aspect_text_char_vocab, char_vocab, char_w2v)
    np.save(os.path.join(folder, 'word_w2v.npy'), word_w2v)
    np.save(os.path.join(folder, 'aspect_word_w2v.npy'), aspect_word_w2v)
    np.save(os.path.join(folder, 'aspect_text_word_w2v.npy'), aspect_text_word_w2v)
    np.save(os.path.join(folder, 'char_w2v.npy'), char_w2v)
    np.save(os.path.join(folder, 'aspect_char_w2v.npy'), aspect_char_w2v)
    np.save(os.path.join(folder, 'aspect_text_char_w2v.npy'), aspect_text_char_w2v)

    print('finished preparing embedding!')
    print('shape of word_w2v:', word_w2v.shape)
    print('sample of word_w2v:', word_w2v[:2, :5])
    print('shape of char_w2v:', char_w2v.shape)
    print('sample of char_w2v:', char_w2v[:2, :5])
    print('shape of aspect_word_w2v:', aspect_word_w2v.shape)
    print('sample of aspect_word_w2v:', aspect_word_w2v[:2, :5])
    print('shape of aspect_char_w2v:', aspect_char_w2v.shape)
    print('sample of aspect_char_w2v:', aspect_char_w2v[:2, :5])
    print('shape of aspect_text_word_w2v:', aspect_text_word_w2v.shape)
    print('sample of aspect_text_word_w2v:', aspect_text_word_w2v[:2, :5])
    print('shape of aspect_text_char_w2v:', aspect_text_char_w2v.shape)
    print('sample of aspect_text_char_w2v:', aspect_text_char_w2v[:2, :5])

    if is_en:
        word_glove = build_glove_embedding(word_vocab, glove_vectors, glove_embed_dim)
        aspect_word_glove = build_asp_embedding(aspect_vocab, word_cut_func, word_vocab, word_glove)
        aspect_text_word_glove = build_asp_text_embedding(aspect_text_word_vocab, word_vocab, word_glove)
        np.save(os.path.join(folder, 'word_glove.npy'), word_glove)
        np.save(os.path.join(folder, 'aspect_word_glove.npy'), aspect_word_glove)
        np.save(os.path.join(folder, 'aspect_text_word_glove.npy'), aspect_text_word_glove)
        print('shape of word_glove:', word_glove.shape)
        print('sample of word_glove:', word_glove[:2, :5])
        print('shape of aspect_word_glove:', aspect_word_glove.shape)
        print('sample of aspect_word_glove:', aspect_word_glove[:2, :5])
        print('shape of aspect_text_word_glove:', aspect_text_word_glove.shape)
        print('sample of aspect_text_word_glove:', aspect_text_word_glove[:2, :5])

    # prepare input
    print('preparing text input...')
    train_word_input = train_data['word_list'].apply(
        lambda x: [word_vocab.get(word, len(word_vocab)+1) for word in x]).values.tolist()
    train_char_input = train_data['char_list'].apply(
        lambda x: [char_vocab.get(char, len(char_vocab)+1) for char in x]).values.tolist()
    test_word_input = test_data['word_list'].apply(
        lambda x: [word_vocab.get(word, len(word_vocab)+1) for word in x]).values.tolist()
    test_char_input = test_data['char_list'].apply(
        lambda x: [char_vocab.get(char, len(char_vocab)+1) for char in x]).values.tolist()
    pickle_dump(train_word_input, os.path.join(folder, 'train_word_input.pkl'))
    pickle_dump(train_char_input, os.path.join(folder, 'train_char_input.pkl'))
    pickle_dump(test_word_input, os.path.join(folder, 'test_word_input.pkl'))
    pickle_dump(test_char_input, os.path.join(folder, 'test_char_input.pkl'))
    print('finished preparing text input!')
    print('length analysis of text word input:')
    analyze_len_distribution(train_word_input, test_word_input)
    print('length analysis of text char input')
    analyze_len_distribution(train_char_input, test_char_input)

    print('preparing aspect input...')
    train_aspect_input = train_data['aspect'].apply(lambda x: [aspect_vocab[x]]).values.tolist()
    test_aspect_input = test_data['aspect'].apply(lambda x: [aspect_vocab[x]]).values.tolist()
    pickle_dump(train_aspect_input, os.path.join(folder, 'train_aspect_input.pkl'))
    pickle_dump(test_aspect_input, os.path.join(folder, 'test_aspect_input.pkl'))
    print('finished preparing aspect input!')

    print('preparing aspect text input...')
    train_aspect_text_word_input = train_data['aspect_word_list'].apply(
        lambda x: [aspect_text_word_vocab.get(word, len(aspect_text_word_vocab) + 1) for word in x]).values.tolist()
    train_aspect_text_char_input = train_data['aspect_char_list'].apply(
        lambda x: [aspect_text_char_vocab.get(char, len(aspect_text_char_vocab) + 1) for char in x]).values.tolist()
    test_aspect_text_word_input = test_data['aspect_word_list'].apply(
        lambda x: [aspect_text_word_vocab.get(word, len(aspect_text_word_vocab) + 1) for word in x]).values.tolist()
    test_aspect_text_char_input = test_data['aspect_char_list'].apply(
        lambda x: [aspect_text_char_vocab.get(char, len(aspect_text_char_vocab) + 1) for char in x]).values.tolist()
    pickle_dump(train_aspect_text_word_input, os.path.join(folder, 'train_word_aspect_input.pkl'))
    pickle_dump(train_aspect_text_char_input, os.path.join(folder, 'train_char_aspect_input.pkl'))
    
    
    pickle_dump(test_aspect_text_word_input, os.path.join(folder, 'test_word_aspect_input.pkl'))
    
    print('finished preparing aspect text input!')
    print('length analysis of aspect text word input:')
    analyze_len_distribution(train_aspect_text_word_input, test_aspect_text_word_input)
    print('length analysis of aspect text char input')
    analyze_len_distribution(train_aspect_text_char_input, test_aspect_text_char_input)

    if 'from' in train_data.columns:
        print('preparing left text input, right text input & position input...')
        train_word_input_l, train_word_input_r, train_word_input_r_with_pad, train_word_mask, train_word_pos_input, \
            train_word_offset_input, train_char_input_l, train_char_input_r, train_char_input_r_with_pad, \
            train_char_mask, train_char_pos_input, train_char_offset_input = split_and_get_loc_info(train_data,
                                                                                                         word_vocab,
                                                                                                         char_vocab,
                                                                                                         word_cut_func)
        pickle_dump(train_word_input_l, os.path.join(folder, 'train_word_input_l.pkl'))
        pickle_dump(train_word_input_r, os.path.join(folder, 'train_word_input_r.pkl'))
        pickle_dump(train_word_input_r_with_pad, os.path.join(folder, 'train_word_input_r_with_pad.pkl'))
        pickle_dump(train_word_mask, os.path.join(folder, 'train_word_mask.pkl'))
        pickle_dump(train_word_pos_input, os.path.join(folder, 'train_word_pos_input.pkl'))
        pickle_dump(train_word_offset_input, os.path.join(folder, 'train_word_offset_input.pkl'))
        pickle_dump(train_char_input_l, os.path.join(folder, 'train_char_input_l.pkl'))
        pickle_dump(train_char_input_r, os.path.join(folder, 'train_char_input_r.pkl'))
        pickle_dump(train_char_input_r_with_pad, os.path.join(folder, 'train_char_input_r_with_pad.pkl'))
        pickle_dump(train_char_mask, os.path.join(folder, 'train_char_mask.pkl'))
        pickle_dump(train_char_pos_input, os.path.join(folder, 'train_char_pos_input.pkl'))
        pickle_dump(train_char_offset_input, os.path.join(folder, 'train_char_offset_input.pkl'))


        test_word_input_l, test_word_input_r, test_word_input_r_with_pad, test_word_mask, test_word_pos_input, \
            test_word_offset_input, test_char_input_l, test_char_input_r, test_char_input_r_with_pad, test_char_mask, \
            test_char_pos_input, test_char_offset_input = split_and_get_loc_info(test_data, word_vocab,
                                                                                      char_vocab, word_cut_func)
        pickle_dump(test_word_input_l, os.path.join(folder, 'test_word_input_l.pkl'))
        pickle_dump(test_word_input_r, os.path.join(folder, 'test_word_input_r.pkl'))
        pickle_dump(test_word_input_r_with_pad, os.path.join(folder, 'test_word_input_r_with_pad.pkl'))
        pickle_dump(test_word_mask, os.path.join(folder, 'test_word_mask.pkl'))
        pickle_dump(test_word_pos_input, os.path.join(folder, 'test_word_pos_input.pkl'))
        pickle_dump(test_word_offset_input, os.path.join(folder, 'test_word_offset_input.pkl'))
        pickle_dump(test_char_input_l, os.path.join(folder, 'test_char_input_l.pkl'))
        pickle_dump(test_char_input_r, os.path.join(folder, 'test_char_input_r.pkl'))
        pickle_dump(test_char_input_r_with_pad, os.path.join(folder, 'test_char_input_r_with_pad.pkl'))
        pickle_dump(test_char_mask, os.path.join(folder, 'test_char_mask.pkl'))
        pickle_dump(test_char_pos_input, os.path.join(folder, 'test_char_pos_input.pkl'))
        pickle_dump(test_char_offset_input, os.path.join(folder, 'test_char_offset_input.pkl'))

        print('length analysis of left text word input:')
        analyze_len_distribution(train_word_input_l, test_word_input_l)
        print('length analysis of left text char input')
        analyze_len_distribution(train_char_input_l, test_char_input_l)
        print('length analysis of right text word input:')
        analyze_len_distribution(train_word_input_r, test_word_input_r)
        print('length analysis of right text char input')
        analyze_len_distribution(train_char_input_r, test_char_input_r)

    # prepare output
    print('preparing output....')
    pickle_dump(train_data['sentiment'].values.tolist(), os.path.join(folder, 'train_label.pkl'))
    
    if 'sentiment' in test_data.columns:
        pickle_dump(test_data['sentiment'].values.tolist(), os.path.join(folder, 'test_label.pkl'))
    print('finished preparing output!')
    print('class analysis of training set:')
    analyze_class_distribution(train_data['sentiment'].values.tolist())

    if 'sentiment' in test_data.columns:
        print('class analysis of test set:')
        analyze_class_distribution(test_data['sentiment'].values.tolist())

In [16]:
config = Config()
glove_vectors, glove_embed_dim = load_glove('data/glove.42B.300d.txt')

In [17]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [18]:
pre_process('data', lambda x: nltk.word_tokenize(x), True)

preprocessing:  data
size of training set: 22428
size of test set: 9612
building vocabulary...
finished building vocabulary!
len of word vocabulary: 20036
sample of word vocabulary: [('portability', 1), ('la', 2), ('islampur', 3), ('historic', 4), ('chandiroor', 5), ('discoveries', 6), ('cum', 7), ('u+0080', 8), ('librarian', 9), ('netaji', 10)]
len of char vocabulary: 55
sample of char vocabulary: [(' ', 1), ('k', 2), ('h', 3), ('f', 4), ('<', 5), ('+', 6), ('!', 7), ('i', 8), ('m', 9), ('^', 10)]
len of aspect vocabulary: 480
sample of aspect vocabulary: [('rbi', 0), ('rape', 1), ('israel', 2), ('paris', 3), ('legislative', 4), ('planes', 5), ('edible', 6), ('reactor', 7), ('road', 8), ('yojana', 9)]
len of aspect text word vocabulary: 480
sample of aspect text word vocabulary: [('rbi', 1), ('rape', 2), ('israel', 3), ('paris', 4), ('legislative', 5), ('planes', 6), ('edible', 7), ('reactor', 8), ('road', 9), ('yojana', 10)]
len of aspect text char vocabulary: 26
sample of aspect tex

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  This is separate from the ipykernel package so we can avoid doing imports until


embedding out of vocabulary： 0
aspect embedding out of vocabulary: 0
aspect text embedding out of vocabulary: 0
embedding out of vocabulary： 0
aspect embedding out of vocabulary: 0
aspect text embedding out of vocabulary: 0
finished preparing embedding!
shape of word_w2v: (20038, 300)
sample of word_w2v: [[ 0.          0.          0.          0.          0.        ]
 [-0.18587156 -0.5147417  -0.06109993 -0.12709035  0.57431316]]
shape of char_w2v: (57, 300)
sample of char_w2v: [[ 0.          0.          0.          0.          0.        ]
 [ 0.04808243  0.03608881 -0.04355727 -0.03675759 -0.14874372]]
shape of aspect_word_w2v: (480, 300)
sample of aspect_word_w2v: [[ 0.20162572 -0.20056358  0.29159245 -0.28464085 -0.14836526]
 [-0.36700869 -0.37976828 -0.404596   -0.29997861 -0.1297816 ]]
shape of aspect_char_w2v: (480, 300)
sample of aspect_char_w2v: [[ 0.03570366  0.0483878   0.00361275 -0.03691958 -0.03800088]
 [ 0.01887818  0.08865483 -0.01886964 -0.06716267 -0.05968153]]
shape of 